In [1]:
setwd("~/0-workspace/CCR7_DC/oral-tolerance-integrate/")

suppressPackageStartupMessages({
library(Seurat) # v4.4
library(Matrix)
library(future)
library(dplyr)
library(ggplot2)
library(cowplot)
library(ComplexHeatmap)
library(circlize)
library(ggrepel)
})
set.seed(1)
options(future.globals.maxSize = Inf)

# pal <- readRDS("plots/palette.rds")
source("utils.R")

In [2]:
cols <- rev(RColorBrewer::brewer.pal(11,"Spectral"))
transitions <- c(0, 15, 25, 30, 40, 50, 60, 70, 75, 85, 100)
scaled_transitions <- scales::rescale(transitions, from = c(0, 100), to = c(0, 1))
scale.color <- scale_color_gradientn(colors = cols, values = scaled_transitions)

In [3]:
RenameGenesSeurat <- function(obj, newnames) { # Replace gene names in different slots of a Seurat object. Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data.
  print("Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data.")
  RNA <- obj@assays$RNA
  
  if (nrow(RNA) == length(newnames)) {
    if (length(RNA@counts)) RNA@counts@Dimnames[[1]] <- newnames
    if (length(RNA@data)) RNA@data@Dimnames[[1]] <- newnames
    if (length(RNA@scale.data)) rownames(RNA@scale.data) <- newnames
    if (length(RNA@meta.features)) {
      RNA@meta.features$prev.names <- rownames(RNA@meta.features)
      rownames(RNA@meta.features) <- newnames
    }
    if (length(RNA@var.features)){
      RNA@var.features <- rownames(RNA@meta.features[match(RNA@var.features, RNA@meta.features$prev.names), ])
    }
  } else {"Unequal gene sets: nrow(RNA) != nrow(newnames)"}
  obj@assays$RNA <- RNA
  return(obj)
}

In [4]:
select.markers <- function(fn, output.dir = 'results/markers/', pairwise = F, fc.thr = 1.5, apv.thr = 0.01, n = Inf, exCC = F, cc.genes = NULL){
  markers <- read.csv(file = paste0(output.dir, fn, ".csv"), header = T, stringsAsFactors = F) %>%
    subset(!grepl("(^MT-)|(^RPS)|(^RPL)|(^MRPL)|(^MRPS)", toupper(gene)))
  if (exCC){
    markers <- markers[!(toupper(markers$gene) %in% toupper(cc.genes)),]
  }
  if(pairwise){
    markers <- subset(markers, abs(avg_log2FC) > log2(fc.thr) & p_val_adj < apv.thr)
    markers$cluster <- ifelse(markers$avg_log2FC > 0, "up", "down")
    if(!is.infinite(n)) markers <- markers %>% group_by(cluster) %>% top_n(n, abs(avg_log2FC))
    markers <- markers[order(markers$avg_log2FC), ]
  } else{
    markers <- subset(markers, avg_log2FC > log2(fc.thr) & p_val_adj < apv.thr)
    if(any(duplicated(markers$gene))) markers <- markers %>% group_by(gene) %>% top_n(1, avg_log2FC)
    if(!is.infinite(n)) markers <- markers %>% group_by(cluster) %>% top_n(n, avg_log2FC)
    markers <- markers[order(markers$cluster, -markers$avg_log2FC), ]
  }
  return(markers)
}

# Load SRO

In [5]:
# load SRO ####
sro.ln <- readRDS("../TC_all_LN/results/SRO_subset.rds")

In [6]:
pref.sro <- "integrate-TC/cca/"; pref.p.sro <- "plots/integrate-ILC3-TC-v3/cca/"
sro <- readRDS(paste0(pref.sro, "SRO.rds"))
sro <- RenameGenesSeurat(sro, toupper(rownames(sro)))

[1] "Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data."


In [7]:
DefaultAssay(sro) <- "RNA"

In [8]:
head(rownames(sro))

[1] "XKR4"   "RP1"    "SOX17"  "MRPL15" "LYPLA1" "TCEA1"

## TC module scores

In [9]:
pref.p.sro
dir.create(paste0(pref.p.sro, "module-scores"))

[1] "plots/integrate-ILC3-TC-v3/cca/"

In [10]:
# load markers ####
markers.ln <- select.markers("TC-clusters", output.dir = paste0('../TC_all_LN/results/markers/'))

In [11]:
tc1.markers.ln <- markers.ln[markers.ln$cluster == 3,]
tc2.markers.ln <- markers.ln[markers.ln$cluster == 2,]
tc3.markers.ln <- markers.ln[markers.ln$cluster == 1,]
tc4.markers.ln <- markers.ln[markers.ln$cluster == 5,]
ki_tc.markers.ln <- markers.ln[markers.ln$cluster == 7,]
transitional.markers.ln <- markers.ln[markers.ln$cluster == 4,]

In [12]:
sum(toupper(tc1.markers.ln$gene) %in% rownames(sro))
sum(toupper(tc2.markers.ln$gene) %in% rownames(sro))
sum(toupper(tc3.markers.ln$gene) %in% rownames(sro))
sum(toupper(tc4.markers.ln$gene) %in% rownames(sro))
sum(toupper(ki_tc.markers.ln$gene) %in% rownames(sro))
sum(toupper(transitional.markers.ln$gene) %in% rownames(sro))

[1] 152

[1] 153

[1] 199

[1] 217

[1] 289

[1] 83

In [13]:
sro <- AddModuleScore(sro, list(toupper(transitional.markers.ln$gene)), name = "transitional.markers")
sro <- AddModuleScore(sro, list(toupper(ki_tc.markers.ln$gene)), name = "ki_tc.markers")
sro <- AddModuleScore(sro, list(toupper(tc1.markers.ln$gene)), name = "tc1.markers")
sro <- AddModuleScore(sro, list(toupper(tc2.markers.ln$gene)), name = "tc2.markers")
sro <- AddModuleScore(sro, list(toupper(tc3.markers.ln$gene)), name = "tc3.markers")
sro <- AddModuleScore(sro, list(toupper(tc4.markers.ln$gene)), name = "tc4.markers")

In [14]:
f.w <- 12; f.h <- 10

In [16]:
pdf(paste0(pref.p.sro, "module-scores/UMAP-transitional_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("transitional.markers1")), val.name = 'module\nscore') + ggtitle("transitional.markers"))
dev.off()

pdf(paste0(pref.p.sro, "module-scores/UMAP-TCI_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("tc1.markers1")), val.name = 'module\nscore') + ggtitle("TC I markers"))
dev.off()

pdf(paste0(pref.p.sro, "module-scores/UMAP-TCII_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("tc2.markers1")), val.name = 'module\nscore') + ggtitle("TC II markers"))
dev.off()

pdf(paste0(pref.p.sro, "module-scores/UMAP-TCIII_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("tc3.markers1")), val.name = 'module\nscore') + ggtitle("TC III markers"))
dev.off()

pdf(paste0(pref.p.sro, "module-scores/UMAP-TCIV_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("tc4.markers1")), val.name = 'module\nscore') + ggtitle("TC IV markers"))
dev.off()

pdf(paste0(pref.p.sro, "module-scores/UMAP-KI_TC_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("ki_tc.markers1")), val.name = 'module\nscore') + ggtitle("Ki67+TC markers"))
dev.off()

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

In [17]:
module.scores <- sro@meta.data[grep("markers1", colnames(sro@meta.data), value = T)]

In [18]:
dir.create(paste0(pref.sro, "module-scores/"))

In [19]:
write.csv(module.scores, paste0(pref.sro, "module-scores/TC_all_LN-scores.csv"), quote = F)